In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

In [2]:
gbif_r = pd.read_csv('gbif_r.csv')

In [3]:
# New dataframe with the lat-long mean and median per year and specie
dif = pd.DataFrame(gbif_r.groupby(['year', 'species'])['lat'].mean())
dif = dif.rename(columns = {'lat': 'lat_mean'})
dif['long_mean'] = gbif_r.groupby(['year', 'species'])['long'].mean()
dif['long_med'] = gbif_r.groupby(['year', 'species'])['long'].median()
dif['lat_med'] = gbif_r.groupby(['year', 'species'])['lat'].median()
dif['regions'] = gbif_r.groupby(['year', 'species'])['region'].agg(lambda x:x.value_counts().index[0])

dif.reset_index(inplace = True)  

In [11]:
# create an empty dataframe
new = pd.DataFrame()

# select species in common in both study years
species20 = list(dif.species[dif['year'] == 2020])
species01 = list(dif.species[dif['year'] == 2001])

spec = list(set(species01).intersection(species20))

# sort the common species
spec.sort()

# set species as the index in our dataframe
new = pd.DataFrame(index=spec)


### AÑOS FIJOS
for s in new.index:
    dif_spec = dif[dif["species"] == s]

    aux_y1 = dif_spec.lat_mean[dif_spec['year'] == 2001].iloc[0]
    aux_y2 = dif_spec.lat_mean[dif_spec['year'] == 2020].iloc[0]

    new.loc[s, 'lat_mean_01-20'] = aux_y2 - aux_y1

new

,lat_mean_01-20
Accipiter gentilis,-0.363295
Accipiter nisus,0.050245
Acrocephalus arundinaceus,-0.485551
Acrocephalus melanopogon,-0.918729
Acrocephalus paludicola,-0.098263
...,...
Upupa epops,1.098976
Uria aalge,-0.059705
Vanellus vanellus,-0.925168
Xema sabini,-0.072182


In [12]:
### AÑOS BUCLE  -- retorna 190 columnas solo para las combinaciones de lat_mean
new = pd.DataFrame()

species20 = list(dif.species[dif['year'] == 2020])
species01 = list(dif.species[dif['year'] == 2001])

spec = list(set(species01).intersection(species20))
spec.sort()
new = pd.DataFrame(index=spec)

years = dif["year"].unique()
years_perm = list(combinations(years, 2))

for s in new.index:
    for y1, y2 in years_perm:
        # get only curren specie
        dif_spec = dif[dif["species"] == s]

        # get lat mean for current specie and years 1 and 2 respectively
        aux_y1 = dif_spec.lat_mean[dif_spec['year'] == y1]
        aux_y2 = dif_spec.lat_mean[dif_spec['year'] == y2]

        # check that current specie has any value in the observed years
        if not(aux_y1.empty or aux_y2.empty):  # if not(aux_y1.empty) and not(aux_y2.empty):
            diff_s = aux_y2.iloc[0] - aux_y1.iloc[0]

        new.loc[s, 'lat_mean_{}-{}'.format(y1, y2)] = diff_s

print(new)

                           lat_mean_2001-2002  lat_mean_2001-2003  \
Accipiter gentilis                  -0.265561           -0.695935   
Accipiter nisus                     -1.462369           -1.159732   
Acrocephalus arundinaceus           -1.731850           -1.336199   
Acrocephalus melanopogon            -0.467446           -0.323003   
Acrocephalus paludicola              1.452525            0.173348   
...                                       ...                 ...   
Upupa epops                         -1.297220           -1.248766   
Uria aalge                          -0.023064           -0.321014   
Vanellus vanellus                   -0.884581           -0.382508   
Xema sabini                          0.564927            0.113151   
Xenus cinereus                       1.996842            1.426485   

                           lat_mean_2001-2004  lat_mean_2001-2005  \
Accipiter gentilis                  -0.730790           -0.266790   
Accipiter nisus                  

# A PARTIR DE AQUÍ TODO SON PRUEBAS

plt.figure(figsize = (13,3))
sns.kdeplot(x = gbif_r.year, color = 'r');

plt.figure(figsize = (15,12))
plot_order = sns.histplot(y = gbif_r.region, 
                          hue = gbif_r.order,
                           multiple = "fill",
                         palette = 'Paired');

plot_order.set_title('Bird distribution by order and regions')
plot_order.set(xlabel = 'Order', 
                ylabel='Region');

funcion que compare de cada especie-año la long /lat ... y me devuelva una lista de
aquellas especies que tengan una diferencia de valor mayor a X 
o que me ordene la diferencia de valor encontrada, y yo hacer un sort y trabajar con las mayores diferencias


# función base con Sonia
nombrecolumna = list(dif.columns)
species = list(dif.species.unique()) 
def compare(columnname, startyear, endyear):
    lista = []
    for s in species:
        a = dif.loc[(dif['species'] == s) & (dif['year'] == startyear)]
        b = dif.loc[(dif['species'] == s) & (dif['year'] == endyear)]
        c = b - a
        lista.append({s: c})
        
    return lista 

### buscar los valores más altos. ABS!!! 

# revisar

def plot_df(df, x, y, title="", xlabel='Date', ylabel='Value', dpi=100):
    plt.figure(figsize=(16,5), dpi = dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

plot_df(df = gbif_r, 
        x = gbif_r.date, 
        y = gbif_r.observations, 
        title = 'Bird ')    

In [ ]:
# Checking changes in the geographic distribution of species
mobility = []
for s in dif.species:
    # subsetear para specie unica 
    #nunique = len + unique?!
    a = dif[dif.species == s]
    if len(a.regions.unique()) != 1:
        mobility.append(s)
    else:
        pass
print(set(mobility))


In [22]:
mobility = []
species_unique = dif.species.unique()
for s in species_unique:
    
   # subsetear para specie unica
   # nunique = len + unique?!
    dif_spec = dif[dif["species"] == s]
        if dif_spec.regions.nunique() > 1:
            mobility.append(s)
        else:
            pass
print(len(mobility))

###################

        # get lat mean for current specie and years 1 and 2 respectively
        aux_y1 = dif_spec.lat_mean[dif_spec['year'] == y1]
        aux_y2 = dif_spec.lat_mean[dif_spec['year'] == y2]

        # check that current specie has any value in the observed years
        if not(aux_y1.empty or aux_y2.empty):  # if not(aux_y1.empty) and not(aux_y2.empty):
            diff_s = aux_y2.iloc[0] - aux_y1.iloc[0]

        new.loc[s, 'lat_mean_{}-{}'.format(y1, y2)] = diff_s

3269


In [19]:
type(dif_spec)

pandas.core.frame.DataFrame

In [ ]:
#  Usar Poisson para describir el número de ocurrencias en un determinado espacio de tiempo